## Stock Scanner

This notebook uses the Trading View API to scan for stocks that could be in play. We use Stochastic RSI to assess whether the fast indicator has risen past the slow indicator to reflect a shift in momentum. If both this shift occurs on both daily and weekly timescales, we say that the stock might be in play

Uses the Trading View TA package: https://python-tradingview-ta.readthedocs.io/en/latest/overview.html

pip install tradingview-ta

In [127]:
from tradingview_ta import *
import pandas as pd
from os.path import exists
import sys

In [128]:
# retrieve a list of all of the symbols in the NYSE
filename = 'nyse-listed_csv.csv'

if exists(filename) == False:
    !wget https://pkgstore.datahub.io/core/nyse-other-listings/nyse-listed_csv/data/3c88fab8ec158c3cd55145243fe5fcdf/nyse-listed_csv.csv

In [129]:
# create a pandas df from the list of files  
nyse_df = pd.read_csv(filename)
print(nyse_df.shape)

(3298, 2)


In [130]:
# retrieve a list of all of the symbols in the NASDAQ
filename = 'nasdaq_screener_1677553848191.csv'

nasdaq_df = pd.read_csv(filename)
print(nasdaq_df.shape)

(7957, 11)


In [131]:
testsyms = ['TSLA','AAPL','MSFT','JGSSTFSD']

In [132]:
# just load all of the stockes into the list 
TVsyms_nyse = list(nyse_df["ACT Symbol"]) 
TVsyms_nas = list(nasdaq_df["Symbol"])

In [ ]:
%%time

# look for stocks matching predefined criteria on daily bars
RSIlow = 30.
RSIhi = 80.
inplay_nyse = []
inplay_nas = []

print("Reading from NYSE:")
for sym in TVsyms_nyse:
    
    try: 
        handle = TA_Handler(
                symbol=sym,
                screener="america",
                exchange="NYSE",
                interval=Interval.INTERVAL_1_DAY
                )
        anal = handle.get_analysis()
        
        # make sure that it is not overbought
        if anal.indicators["RSI"] <= RSIhi:
            # fast momentum is Stoch.K, and slow momentum is Stoch.D
            if anal.indicators['Stoch.K'] > anal.indicators['Stoch.D']:
                inplay_nyse.append(sym)
    except Exception:
        pass

print("Reading from NASDAQ:")
for sym in TVsyms_nas:
    
    try: 
        handle = TA_Handler(
                symbol=sym,
                screener="america",
                exchange="masdaq",
                interval=Interval.INTERVAL_1_DAY
                )
        anal = handle.get_analysis()
        
        # make sure that it is not overbought
        if anal.indicators["RSI"] <= RSIhi:
            # fast is Stoch.K, and slow is Stoch.D
            #print(sym,anal.indicators["RSI"],\
            #      anal.indicators['Stoch.K'],anal.indicators['Stoch.D'])
            if anal.indicators['Stoch.K'] > anal.indicators['Stoch.D']:
                inplay_nas.append(sym)
    except Exception:
        pass
    

Reading from NYSE:


In [ ]:
print(len(inplay_nyse + inplay_nas))

In [ ]:
%%time 

# now look for stocks matching predefined criteria on weekly bars
inplaywkly_nyse = []
inplaywkly_nas = []
volwkly_nyse = []
volwkly_nas = []
price_nyse = []
price_nas = []
stochk_nyse = []
stochk_nas = []
stochd_nyse = []
stochd_nas = []
rsi_nyse = []
rsi_nas = []


print("Checking weekly NYSE data:")
for sym in inplay_nyse:
    
    try: 
        handle = TA_Handler(
                symbol=sym,
                screener="america",
                exchange="NYSE",
                interval=Interval.INTERVAL_1_WEEK
                )
        anal = handle.get_analysis()
        
        if anal.indicators["RSI"] <= RSIhi:
            # fast is Stoch.K, and slow is Stoch.D
            #print(sym,anal.indicators["RSI"],\
            #      anal.indicators['Stoch.K'],anal.indicators['Stoch.D'])
            if anal.indicators['Stoch.K'] > anal.indicators['Stoch.D']:
                inplaywkly_nyse.append(sym)
                volwkly_nyse.append(anal.indicators['volume'])
                price_nyse.append(anal.indicators['close'])
                stochk_nyse.append(anal.indicators['Stoch.K'])
                stochd_nyse.append(anal.indicators['Stoch.D'])
                rsi_nyse.append(anal.indicators['RSI'])
    except Exception:
        pass

print("Checking weekly NASDAQ data:")
for sym in inplay_nas:
    
    try: 
        handle = TA_Handler(
                symbol=sym,
                screener="america",
                exchange="nasdaq",
                interval=Interval.INTERVAL_1_WEEK
                )
        anal = handle.get_analysis()
        
        if anal.indicators["RSI"] <= RSIhi:
            # fast momentum is Stoch.K, and slow momentum is Stoch.D
            if anal.indicators['Stoch.K'] > anal.indicators['Stoch.D']:
                inplaywkly_nas.append(sym)
                volwkly_nas.append(anal.indicators['volume'])
                price_nas.append(anal.indicators['close'])
                stochk_nas.append(anal.indicators['Stoch.K'])
                stochd_nas.append(anal.indicators['Stoch.D'])
                rsi_nas.append(anal.indicators['RSI'])
    except Exception:
        pass



In [ ]:
inplaywkly = inplaywkly_nyse + inplaywkly_nas
volwkly = volwkly_nyse + volwkly_nas
price = price_nyse + price_nas
stochk = stochk_nyse + stochk_nas
stochd = stochd_nyse + stochd_nas
rsi = rsi_nyse + rsi_nas

df = pd.DataFrame(
    {'SYM': inplaywkly,
     'VOL': volwkly,
     'PRICE': price,
     'StochK': stochk,
     'StochD': stochd,
     'RSI': rsi
    })


In [ ]:
lowrsi = df[(df["RSI"] < 100.) & (df["VOL"] > 100000.) & (df['PRICE'] < 500.) & (df['PRICE'] > 2.)]
pd.set_option('display.max_rows', None)
print(lowrsi.sort_values(by=['PRICE', 'VOL'], ascending=False))